In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import glob
from pathlib import Path
import os

import time

# ---- Imports from your imputer ----
from graph_imputer import (
    iterative_graph_imputation_final,
    compute_biavailability_matrix,
    compute_spearman_matrix,
)


missingness = 'mar'

In [2]:
def evaluate_imputations(df_experimental: pd.DataFrame,
                         df_imputed: pd.DataFrame,
                         df_ground_truth: pd.DataFrame,
                         return_arrays: bool = False):
    """
    For each column, find rows where df_experimental is NaN (i.e., values were imputed),
    extract the corresponding values from df_imputed and df_ground_truth as arrays,
    and compute R^2, Pearson r, RMSE, and MAE between those arrays.

    Returns:
        metrics_df: pd.DataFrame indexed by column name with columns:
                    ["R2", "Pearson_r", "RMSE", "MAE", "n_imputed", "n_scored"]
        arrays_by_col (optional): dict[col] -> {"imputed": np.ndarray, "ground_truth": np.ndarray}
                                  Arrays are the pairs actually used for scoring (after dropping any stray NaNs).
    """
    # Basic alignment checks
    if not (df_experimental.columns.equals(df_imputed.columns) and
            df_experimental.columns.equals(df_ground_truth.columns)):
        # Align on shared columns in the same order
        common_cols = df_experimental.columns.intersection(df_imputed.columns).intersection(df_ground_truth.columns)
        df_experimental = df_experimental[common_cols].copy()
        df_imputed = df_imputed[common_cols].copy()
        df_ground_truth = df_ground_truth[common_cols].copy()

    rows = []
    arrays_by_col = {}

    # Only score numeric columns
    numeric_cols = [c for c in df_experimental.columns if pd.api.types.is_numeric_dtype(df_experimental[c])]

    for col in numeric_cols:
        mask_imputed = df_experimental[col].isna()
        n_imputed = int(mask_imputed.sum())

        if n_imputed == 0:
            rows.append({
                "column": col,
                "R2": np.nan,
                "Pearson_r": np.nan,
                "RMSE": np.nan,
                "MAE": np.nan,
                "n_imputed": 0,
                "n_scored": 0
            })
            continue

        imp_vals = df_imputed.loc[mask_imputed, col].to_numpy(dtype=float, copy=False)
        gt_vals  = df_ground_truth.loc[mask_imputed, col].to_numpy(dtype=float, copy=False)

        # Drop any pairs with NaNs in either array (in case of upstream issues)
        valid = (~np.isnan(imp_vals)) & (~np.isnan(gt_vals))
        imp_vals = imp_vals[valid]
        gt_vals  = gt_vals[valid]
        n_scored = int(valid.sum())

        if return_arrays:
            arrays_by_col[col] = {"imputed": imp_vals.copy(), "ground_truth": gt_vals.copy()}

        if n_scored == 0:
            rows.append({
                "column": col,
                "R2": np.nan,
                "Pearson_r": np.nan,
                "RMSE": np.nan,
                "MAE": np.nan,
                "n_imputed": n_imputed,
                "n_scored": 0
            })
            continue

        # Metrics
        # R^2: sklearn defines R^2 = 0.0 for constant y_true; that's acceptable here.

        if np.var(gt_vals) < 1e-8:
            R2 = 0
        else:
            R2 = r2_score(gt_vals, imp_vals)

        # Pearson r requires at least 2 points and non-zero std in both arrays
        if n_scored >= 2 and np.std(imp_vals) > 0 and np.std(gt_vals) > 0:
            Pearson_r = float(pearsonr(gt_vals, imp_vals).statistic)
        else:
            Pearson_r = np.nan

        RMSE = np.sqrt(float(mean_squared_error(gt_vals, imp_vals)))
        MAE  = float(mean_absolute_error(gt_vals, imp_vals))

        rows.append({
            "column": col,
            "R2": R2,
            "Pearson_r": Pearson_r,
            "RMSE": RMSE,
            "MAE": MAE,
            "n_imputed": n_imputed,
            "n_scored": n_scored
        })

    metrics_df = pd.DataFrame(rows).set_index("column").sort_index()

    if return_arrays:
        return metrics_df, arrays_by_col
    return metrics_df

In [3]:
output_dir = Path(f"hyperparameter_tables/{missingness}")
csv_files = [f for f in output_dir.glob("*.csv") if f.is_file() and "metadata" not in str(f)]
csv_files = [str(f) for f in csv_files]

df_ground_truth = pd.read_csv(f'../datasets/experimental_data/numerical/eeg/eeg_ground_truth.csv').drop('V13', axis=1)
df_experimental = pd.read_csv(f'../datasets/experimental_data/numerical/eeg/{missingness}/eeg_{missingness}_0.1.csv').drop('V13', axis=1)

In [4]:
imputed_dfs = dict()
for imputation_file in csv_files:
    hp_uid = imputation_file.split('/')[-1].split('_')[0]
    imputed_dfs[hp_uid] = pd.read_csv(imputation_file).drop('V13', axis=1)

In [5]:
HP_incomplete = []
for hp_exp in imputed_dfs:
    if imputed_dfs[hp_exp].isna().sum().sum() > 0:
        print(hp_exp)
        HP_incomplete.append(hp_exp)

In [6]:
metadata = pd.read_csv(f'hyperparameter_tables/{missingness}/metadata.csv')

In [15]:
metadata[metadata['uid'].isin(HP_incomplete)].head()

,uid,status,error,runtime_seconds,input_csv,missingness_type,method,prioritization_mode,partner_proportion,edge_threshold_rule,edge_threshold_value,alpha,beta,model_func,max_rounds,output_imputed_csv,output_runtime_txt


In [20]:
mean_summary = dict()
sem_summary = dict()

for hp_exp in imputed_dfs:
    res = evaluate_imputations(df_experimental, imputed_dfs[hp_exp], df_ground_truth)
    mean_summary[hp_exp] = res.mean()
    sem_summary[hp_exp]  = res.sem(numeric_only=True, ddof=1)

/var/folders/7v/1hw0skqj72dczc6xdjymzmyh0000gn/T/ipykernel_85844/1739287539.py:81: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  Pearson_r = float(pearsonr(gt_vals, imp_vals).statistic)
/var/folders/7v/1hw0skqj72dczc6xdjymzmyh0000gn/T/ipykernel_85844/1739287539.py:81: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  Pearson_r = float(pearsonr(gt_vals, imp_vals).statistic)
/var/folders/7v/1hw0skqj72dczc6xdjymzmyh0000gn/T/ipykernel_85844/1739287539.py:81: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  Pearson_r = float(pearsonr(gt_vals, imp_vals).statistic)
/var/folders/7v/1hw0skqj72dczc6xdjymzmyh0000gn/T/ipykernel_85844/1739287539.py:81: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  Pearson_r = float(pearsonr(gt_vals, imp_vals).statistic)
/var/folders/7v/1hw0skqj72dczc6xdjymzmyh0000gn/T/ipykern

In [22]:
res_df = pd.DataFrame(mean_summary).T.sort_values(by='RMSE')
metadata = metadata.set_index('uid')

In [24]:
metadata_res = pd.merge(res_df[['RMSE', 'MAE', 'R2', 'Pearson_r']], 
                         metadata[['runtime_seconds', 'method', 'prioritization_mode', 'partner_proportion',
                                    'edge_threshold_rule', 'edge_threshold_value', 'alpha', 'beta', 'model_func', 'max_rounds']], 
                         left_index=True, right_index=True).sort_values(by='RMSE')

In [11]:
metadata_res.to_csv('hyperparameter_results_MAR.csv')

In [30]:
metadata_res.drop(['MAE', 'R2', 'Pearson_r'], axis=1).to_csv(''

,RMSE,runtime_seconds,method,prioritization_mode,partner_proportion,edge_threshold_rule,edge_threshold_value,alpha,beta,model_func,max_rounds
HP0078,0.007288,2.7153,eigenvector,centrality,0.08,p85,0.465596,1.5,0.5,LinearRegression,1
HP0100,0.007288,50.9655,eigenvector,utility,0.15,p85,0.523886,1.0,1.0,LinearRegression,1
HP0011,0.007288,2.7037,degree,centrality,0.15,p85,0.594056,0.5,1.5,LinearRegression,1
HP0056,0.007288,52.4761,betweenness,utility,0.08,NaN,NaN,0.5,1.5,LinearRegression,1
HP0095,0.007288,51.4912,eigenvector,utility,0.08,p85,0.594056,0.5,1.5,LinearRegression,1
...,...,...,...,...,...,...,...,...,...,...,...
HP0051,0.036028,7.2360,betweenness,centrality,0.25,NaN,NaN,1.5,0.5,LinearRegression,1
HP0049,0.038034,7.1702,betweenness,centrality,0.25,NaN,NaN,1.0,1.0,LinearRegression,1
HP0089,0.053379,7.8892,eigenvector,centrality,0.25,p85,0.594056,0.5,1.5,LinearRegression,1
HP0088,0.053417,7.2745,eigenvector,centrality,0.25,p85,0.523886,1.0,1.0,LinearRegression,1


In [12]:
metadata_res.RMSE.unique()

array([0.00728806, 0.0140043 , 0.01404307, 0.01417506, 0.0141767 ,
       0.01421764, 0.01502793, 0.01506625, 0.01853094, 0.01857948,
       0.01857982, 0.01861802, 0.03602818, 0.03803445, 0.05337885,
       0.05341745])